Write python scripts to answer the following questions:
```
Q1 Who did spend most money for renting?
Q2 Which car does make the most amount of income?
Q3 How many time Erick Gilbert (id=6) rent the cars? (need to query from the name)
Q4 What are the car rented by Ronald Cunningham? (need to query from the name)
Q5 Between 17/03/2014(inclusive) and 29/03/2014(inclusive), 
    what is the total income of ALL cars in the range of time?
Q6 Between 17/03/2014(inclusive) and 29/03/2014(inclusive), 
    what are the total incomes of EACH car in the range of time?
Q7 Find all rental serveices that has duration less than than 3 days?
Q8 Find all rental serveices that has duration greater than or equal than 2 day?
Q9 Find the total rent duration of EACH car?
Q10 Find the ratio [total_income_of_car]/[car_price] of EACH car?
```

```
Q1 ใครจ่ายเงินค่าเช้ามากที่สุด
Q2 รถคันไหนทำงานได้มากที่สุด
Q3 Erick Gilbert (id=6) ยืมรถไปกี่ครั้ง (need to query from the name)
Q4 Ronald Cunningham ยืมรถคันไหนบ้าง (need to query from the name)
Q5 ช่วงวันที่ 17/03/2014(inclusive) ถึง 29/03/2014(inclusive), 
    หารายได้รวมของรถทุกคันในช่วงเวลานี้
Q6 ช่วงวันที่ 17/03/2014(inclusive) ถึง 29/03/2014(inclusive),
    หารายได้รวมของรถแต่ละคันในช่วงเวลานี้
Q7 การเช่ารถครั้งใดบ้างที่มีระยะเวลายืมน้อยกว่า 3 วัน
Q8 การเช่ารถครั้งใดบ้างที่มีระยะเวลายืมมากกว่าหรือเท่ากับ 2 วัน
Q9 หาระยะเวลารวมของการเช่ารถแต่ละคัน
Q10 หาอัตราส่วน [total_income_of_car]/[car_price] ของรถแต่ละคัน
```